## Design your own trigger

The trigger decides what type of physics we record forever. While ATLAS has a suite of inclusive triggers, for example a trigger for events containing at least one electron/positron above 24 GeV or at least one hadronic jet above 400 GeV, there is also room for more specialised triggers that looks for events that look like a specific Standard Model or hypothetical new process. 
Good specialiased triggers do two things:
- They have a good signal-to-noise ratio.
- They select events at a low/affordable rate.

Today, we are interested in the following process, the production of a Higgs boson via Vector Boson Fusion (VBF), which then decays to invisible particles. Decays of Higgs to invisible is an interesting search at ATLAS for new physics.


The current MET trigger selects events containing total missing transverse energy (MET) of at least 200 GeV. Can we use the event topology to design a new trigger that makes our event selection more rich with events containing VBF Higgs$\rightarrow E_T^{miss}$?

<img img align="center" src="https://bpb-us-w2.wpmucdn.com/web.sas.upenn.edu/dist/2/672/files/2020/03/Higgs_invisible_vbf_fig_01a.png" width="400" />

## Trigger limitations

Our trigger will have to fit into the current ATLAS rate budget. We have room for an additional trigger with a rate of 5 Hz. The current 200 GeV single $E_T^{miss}$ trigger has a rate of 35 Hz and a significance ($S/\sqrt{B}$) of 1.31 for the VBF $H\rightarrow E_T^{miss}$ process for 300 inverse femtobarns of collected data. Can we improve that?

| 200 GeV MET Trigger Rate | VBF H trigger Max Allowed Rate | Rel. Reduction |
| --- | --- | --- |
| 35 Hz | 5 Hz | 0.14



### How to design your trigger?

The code below sets up some histograms that may hint at some event features that might show a marked difference between signal (VBF H$\rightarrow E_T^{miss}$) and background (data). You can use these features to apply an event selection that will enhance the signal-to-background ratio.

**Remember**: Additional selections means we can lower the high 200 GeV threshold on the total missing transverse energy!

You should also have a look at the following paper section 4.2 to get inspiration for event selections:
- ["Search for invisible Higgs-boson decays in events with vector-boson fusion signatures using 139 fb−1 of proton-proton data recorded by the ATLAS experiment"](https://arxiv.org/abs/2202.07953)

### The final goal?

Design a trigger that is affordable and has a high significance! A significance of 3 is good, of 5 is very good!

In [ ]:
import ROOT
import math

# -- set up of common variables ---

tree_name = "mini" # event "tree" in which information of each event in the data set are defined:
                   # event level information, particles and their properties

GeV = 1e-3 # convert MeV to GeV 
fb = 1e-3

In [ ]:
# --- list of histograms --- 
# NOTE: Not all histograms are filled in the analysis code. 
# They hint at what could be interesting to look at, though!

bin_width = 5 # GeV
h_pt_min = 0
h_pt_max = 450
pt_bins = int((h_pt_max-h_pt_min)/bin_width)

def get_histograms():
    # photon histograms
    hist_list = {}
    hist_list["met_et"] = ROOT.TH1F("h_met_et","met_et; met ET [GeV]; events", pt_bins,h_pt_min,h_pt_max)

    hist_list["jet_n"] = ROOT.TH1F("h_jet_n","jet multiplicity; jet multiplicity; events", 10, 0, 10)
    hist_list["jet0_pt"]= ROOT.TH1F("h_jet0_pt","leading jet pT; leading jet pT [GeV]; events", pt_bins,h_pt_min,h_pt_max)
    hist_list["jet1_pt"] = ROOT.TH1F("h_jet1_pt","subleading jet pT; subleading jet pT [GeV]; events", pt_bins,h_pt_min,h_pt_max)
    hist_list["jet0_eta"] = ROOT.TH1F("h_jet0_eta","leading jet eta; leading jet eta; events",100,-5,5)
    hist_list["jet1_eta"] = ROOT.TH1F("h_jet1_eta","subleading jet eta; subleading jet eta; events",100,-5,5)
    hist_list["jet_dphi"] = ROOT.TH1F("h_jet_dphi","DeltaPhi(jet0,jet1); DeltaPhi; events", 80, -4, 4)
    hist_list["jet_deta"] = ROOT.TH1F("h_jet_deta","DeltaEta(jet0,jet1); DeltaEta; events", 80, 0, 8)
    hist_list["dijet_m"]= ROOT.TH1F("h_dijet_m","dijet mass; dijet mass [GeV]; events", pt_bins*2,h_pt_min,h_pt_max*2)

    hist_list["cutflow"] = ROOT.TH1F("cutflow", "cutflow;cut;count", 10,0,10)
    
    for hname, hist in hist_list.items():
        hist.SetDirectory(0)

    
    return hist_list

In [ ]:
# ------------------------------
# --- Trigger Selection Code --- 
# ------------------------------
# This is where we implement the event selection of the trigger based on objects and combined object systems.
# Since ATLAS Open Data does not have our process of interest, we first convert the leptons to missing energy,
# to simulate our process. Our main background is Z->nunu.

def analyse_events(tree, hist_list, max_events = -1):
    
    for hname, hist in hist_list.items():
        hist.Reset()
        
    passed_cnt = 0
    evt_cnt = 0
    
    met_thresh = 200

    for event in tree:
        
        if evt_cnt >= max_events: break
        evt_cnt += 1
        ## -- calculating total met - see comment at top of cell -- 
        met_x = 0
        met_y = 0
        for ii in range(event.lep_n):
            met_x += event.lep_pt[ii]*math.cos(event.lep_phi[ii])
            met_y += event.lep_pt[ii]*math.sin(event.lep_phi[ii])
        met_x += event.met_et*math.cos(event.met_phi)
        met_y += event.met_et*math.sin(event.met_phi)
        met_et_GeV = math.sqrt(met_x*met_x + met_y*met_y) * GeV
        # --- 
    
        hist_list["cutflow"].Fill("total",1)
        hist_list["met_et"].Fill(met_et_GeV)

        # our initial MET trigger - can we lower the threshold by using other selections in parallel?
        if met_et_GeV < met_thresh: continue
        hist_list["cutflow"].Fill(f"{met_thresh}GeVMET",1)
        
        # reconstruct jets - could come in handy
        hist_list["jet_n"].Fill(event.jet_n) 
        
        if event.jet_n >= 2:
            jet0 = ROOT.TLorentzVector()
            jet0.SetPtEtaPhiE(event.jet_pt[0], event.jet_eta[0], event.jet_phi[0], event.jet_E[0])
            jet1 = ROOT.TLorentzVector()
            jet1.SetPtEtaPhiE(event.jet_pt[1], event.jet_eta[1], event.jet_phi[1], event.jet_E[1])
        
            jet0_pt_GeV = jet0.Pt()*GeV
            jet1_pt_GeV = jet1.Pt()*GeV
            jet0_eta = jet0.Eta()
            jet1_eta = jet1.Eta() 
            
            hist_list["jet0_pt"].Fill(jet0_pt_GeV)
            hist_list["jet1_pt"].Fill(jet1_pt_GeV)
            hist_list["jet0_eta"].Fill(jet0_eta)
            hist_list["jet1_eta"].Fill(jet1_eta)
        
            
        # <FILL ME>
        
        passed_cnt += 1

        
    return passed_cnt

        

In [ ]:
# ------------------------------
# --- RETRIEVE MC SIGNAL -------
# ------------------------------
### You should not have to change anything here.

signal_hist_list = get_histograms()

base_url = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC"
input_name = "mc_345323.VBFH125_WW2lep.2lep.root"

infile = ROOT.TFile.Open(f"{base_url}/{input_name}")
signal_tree = infile.Get("mini")

In [ ]:
# ------------------------------
# --- ANALYSE MC SIGNAL --------
# ------------------------------
### You should not have to change anything here.

total_evts = signal_tree.GetEntries()

# Get signal cross-section
signal_tree.GetEntry(0)
xsection = signal_tree.XSection * fb
print("Signal x-section = %.4f fb"%(xsection))

# Analyse signal events
max_evts = total_evts # can reduce max_evts for fast histogram analysis but NOTE for final result revert this to total_evts!
scale_factor = total_evts/max_evts
print("Processing %i events..."%(max_evts))        
passed_signal_events = analyse_events(signal_tree, signal_hist_list,max_evts) * scale_factor

print("passed signal events = %.2f"%(passed_signal_events))

# Signal events are corrected by the cross-section.
signal = passed_signal_events * xsection
print("x-section corrected signal = %.2f"%(signal))

In [ ]:
# -----------------------------------
# --- RETRIEVE BACKGROUND DATA ------
# -----------------------------------
### You should not have to change anything here.

bg_hist_list = get_histograms()

base_url = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/Data"
input_name = "data_A.2lep.root"

infile = ROOT.TFile.Open(f"{base_url}/{input_name}")
infile.GetListOfKeys().Print()
bg_tree = infile.Get("mini")

In [ ]:
# -----------------------------------
# --- ANALYSE BACKGROUND DATA -------
# -----------------------------------
### You should not have to change anything here.

total_evts = bg_tree.GetEntries()

# Analyse background events
max_evts = total_evts # can reduce max_evts for fast histogram analysis but NOTE for final result revert this to total_evts!
print("Processing %i events..."%(max_evts))
scale_factor = total_evts/max_evts
passed_bg_events = analyse_events(bg_tree, bg_hist_list, max_evts) * scale_factor
print("passed background events = %i"%(passed_bg_events))

# Background events are scaled by a factor 18 here as we have only analysed 1/18 of the 10 ifb dataset.
background = passed_bg_events * 18
print("scaled background = %i"%(background))

In [ ]:
# ---------------------
# -- Plotting Code ---
# ---------------------
# Here we have a look at the distributions. Which distributions look very different
# for signal (red) and background (blue) events? Can we use those differences to enhance
# the signal selection of our trigger?
# (NOTE: Ignore errors about histogram normalisation. This will happen for histograms we have not filled.)


canvas = ROOT.TCanvas("Canvas", "c", 1200, 800)
canvas.Divide(4,3,0.001,0.001) # -> (3,4): 3 plots per row, 4 rows

cnv = 1
for hname in signal_hist_list:
    signal_hist = signal_hist_list[hname]
    bg_hist = bg_hist_list[hname]
    signal_hist.SetFillColorAlpha(ROOT.kRed, 0.5)
    bg_hist.SetFillColorAlpha(ROOT.kBlue, 0.5)
    canvas.cd(cnv)
    if not "cutflow" in hname:
        bg_hist.DrawNormalized("HIST")
        signal_hist.DrawNormalized("HISTSAME")
    canvas.Draw()
    cnv+=1

In [ ]:
canvas2 = ROOT.TCanvas("Canvas2", "c2", 800, 600)

signal_hist_cp = signal_hist_list["cutflow"].Clone("sg_cutflow_cp")
signal_hist_cp.Scale(1./signal_hist_cp.GetMaximum())
signal_hist_cp.Draw("H")
bg_hist_cp = bg_hist_list["cutflow"].Clone("bg_cutflow_cp")
bg_hist_cp.Scale(1./bg_hist_cp.GetMaximum())
bg_hist_cp.Draw("HSAME")
#ROOT.gPad.SetLogy()
canvas2.Draw()

In [ ]:
# ----------------------
# --- Final Results ----
# ----------------------
# Now we calculate the significance of our signal after selection and our trigger "rate"

import math

reduction = background/51336 # denominator: Number of events of 200 GeV MET trigger
lumi_factor = 30. # 30 * 10 inverse fb = 300 ifb, the total amount expected for LHC Run 3

print("Passed signal events = %i"%signal)
print("Passed bg events = %i"%background)
print("Significance = %.2f "%(signal*lumi_factor/math.sqrt(background*lumi_factor)))
print("Background rate relative to 200 GeV MET trigger = %.3f"%reduction )
if reduction > 0.14:
    print(" -- Sorry, we can't afford this trigger.")
else:
    print(" -- We can afford this trigger if it is worth the physics (good significance)!")